In [1]:
pip install adversarial-robustness-toolbox


     |████████████████████████████████| 655kB 2.8MB/s 
     |████████████████████████████████| 20.8MB 1.2MB/s 
     |████████████████████████████████| 235kB 39.7MB/s 
     |████████████████████████████████| 6.8MB 42.9MB/s 
     |████████████████████████████████| 747kB 39.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [225]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch
import torchvision.models as models
from torchvision import transforms as T

from PIL import Image
from art.attacks.evasion import DeepFool
from art.estimators.classification import PyTorchClassifier
from art.utils import load_mnist


In [272]:
transform = T.Compose([            
 T.Resize(256),                    
 T.CenterCrop(224),                
 T.ToTensor(),                     
 T.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225])
 ])


In [273]:
import urllib
url, filename = ("https://cdn.mos.cms.futurecdn.net/6t8Zh249QiFmVnkQdCCtHK-320-80.jpg", "example.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

img = Image.open('dog.jpeg')
transformed_img = transform(img)
input_img = transformed_img.unsqueeze(0)

In [274]:
arr = []
image_array = np.asarray(img)

x = np.moveaxis(image_array, -1, 0)
arr.append(x)
arr = np.array(arr, dtype = 'float32')

print("Expected",image_array.shape)
print("Current", x.shape)

z =  np.moveaxis(x, 0, -1)
print("Try",z.shape)

Expected (194, 259, 3)
Current (3, 194, 259)
Try (194, 259, 3)


In [275]:
model = models.resnet18(pretrained=True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(0, 1),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 224, 224),
    nb_classes=1000,
)

In [278]:
attack = FastGradientMethod(estimator = classifier)
advx = attack.generate(x=arr)


In [279]:
print(advx[0].shape)
res = np.moveaxis(advx[0], 0 , 2)

output_image = Image.fromarray(res, 'RGB')
output_image.save('dog_FGSM.jpg')

img.show()

(3, 194, 259)


In [214]:
print(advx[0].shape)
print(x.shape)
print(image_array.shape)

(3, 194, 259)
(3, 194, 259)
(194, 259, 3)


#**Classifier for original**


In [163]:
model = models.resnet18(pretrained=True).eval()


In [164]:
transform = T.Compose([            
 T.Resize(256),                    
 T.CenterCrop(224),                
 T.ToTensor(),                     
 T.Normalize(mean=[0.485, 0.456, 0.406],
             std=[0.229, 0.224, 0.225])
 ])


In [165]:
img = Image.open('dog.jpeg')
transformed_img = transform(img)
input_img = transformed_img.unsqueeze(0)

In [166]:
output = model(input_img)
print(output.shape)

torch.Size([1, 1000])


In [167]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

In [168]:
_, indices = torch.sort(output, descending=True)
[(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]


[('kuvasz', 0.12053955346345901),
 ('golden retriever', 0.1469627320766449),
 ('Great Pyrenees', 0.012410273775458336),
 ('Labrador retriever', 0.0342085026204586),
 ('clumber, clumber spaniel', 0.10219232738018036)]

#**Adversarial**

In [169]:
imgx = Image.open('advxxxdog_FGSM.jpg')
transformed_img = transform(img)
input_img = transformed_img.unsqueeze(0)

In [170]:
output = model(input_img)
print(output.shape)

torch.Size([1, 1000])


In [171]:
with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

In [172]:
_, indices = torch.sort(output, descending=True)
[(classes[idx], percentage[idx].item()) for idx in indices[0][:5]]

[('kuvasz', 0.12053955346345901),
 ('golden retriever', 0.1469627320766449),
 ('Great Pyrenees', 0.012410273775458336),
 ('Labrador retriever', 0.0342085026204586),
 ('clumber, clumber spaniel', 0.10219232738018036)]

#*Adversarial + original*